# 1. Generate Question

Let me explain the main input and output flows in `generateQuestion.js`:

### Main Input/Output Flow:

1. **Input (User Interface)**:
```javascript
// From standard form:
{
    topic: string,            // from #topic input
    level: string,           // from #level input
    questionCount: number,   // from #question-count input
    extraRequirements: string // from #extra-requirements input
}

// OR

// From custom form:
{
    customPrompt: string    // from #custom-prompt-text textarea
}
```

2. **API Call Output** (sent to backend):
```javascript
// POST to ${API_URL}/generate-questions
{
    prompt: string  // Either formatted standard prompt or custom prompt
}
```

3. **API Response Input** (received from backend):
```javascript
// Expected format:
[
    {
        question: string,
        structure: string,
        "main phrase": string,
        "optional phrase 1": string,
        "optional phrase 2": string,
        "question-vi": string,
        "structure-vi": string,
        "main phrase-vi": string,
        "optional phrase 1-vi": string,
        "optional phrase 2-vi": string
    },
    // ... more questions
]
```

4. **Final Output** (displayed to user):
- A table displaying all questions with their translations
- Each row has edit and delete functionality
- The data is stored in `storagedLessons` variable for further manipulation

### Key Processing Functions:

```javascript:frontend/src/js/generateQuestion.js
// Input processing
function createGenerateQuestionPrompt() {
    // Converts form inputs into formatted prompt string
}

// Output processing
function processApiResponse(data) {
    // Converts API response into usable lesson format
    // Adds unique lesson_id to each lesson
    // Stores in storagedLessons
}

function displayGeneratedQuestions(lessons) {
    // Renders lessons as HTML table
}
```

The flow is: User Input → API Request → API Response → Processed Data → Display Table


------------
@generateQuestion.js 

---------------------
Muốn tạo @tableDraftTracking.js , file này: 
---------------
1. mỗi lần ấn generatQuestion => sẽ tạo lesson_id  (topic_time_date, chẳng hạn: nametopic_2200_01112024   tạo lúc 22h ngày 1/11/2024)
2. input: là {
    topic: string,            // from #topic input
    level: string,           // from #level input
    questionCount: number,   // from #question-count input
    extraRequirements: string // from #extra-requirements input
}

3. raw 
sẽ là expected format 


    {
        question: string,
        structure: string,
        "main phrase": string,
        "optional phrase 1": string,
        "optional phrase 2": string,
        "question-vi": string,
        "structure-vi": string,
        "main phrase-vi": string,
        "optional phrase 1-vi": string,
        "optional phrase 2-vi": string
    },

4. final sẽ 
là JSON của bảng tạo ra (KHI NGƯỜI DÙNG ẤN Copy table) 
===============
File @tableDraftTracking.js này sẽ có trách nhiệm tạo 4 cái trên
sau đó call tới @tableDraftController.js để lưu data vào Larkbase



Tôi sẽ giúp bạn tạo `tableDraftTracking.js` để theo dõi và lưu trữ quá trình tạo câu hỏi:

```javascript:frontend/src/js/trackings/tableDraftTracking.js
import { config } from '../config.js';

const API_URL = config.production.apiUrl;

class TableDraftTracking {
    static generateLessonId(topic) {
        const now = new Date();
        const time = now.getHours().toString().padStart(2, '0') + 
                    now.getMinutes().toString().padStart(2, '0');
        const date = now.getDate().toString().padStart(2, '0') + 
                    (now.getMonth() + 1).toString().padStart(2, '0') + 
                    now.getFullYear();
        
        // Normalize topic name (remove spaces, special characters)
        const normalizedTopic = topic.toLowerCase()
            .replace(/[^a-z0-9]/g, '_')
            .replace(/_+/g, '_');
            
        return `${normalizedTopic}_${time}_${date}`;
    }

    static async trackDraftGeneration(inputData, rawResponse, finalTable = '') {
        try {
            const lesson_id = this.generateLessonId(inputData.topic);
            
            // Prepare input data
            const input = JSON.stringify({
                topic: inputData.topic,
                level: inputData.level,
                questionCount: inputData.questionCount,
                extraRequirements: inputData.extraRequirements
            });

            // Prepare raw data (API response)
            const raw = JSON.stringify(rawResponse);

            // Final data will be updated later when user clicks "Copy Table"
            const draftData = {
                lesson_id,
                input,
                raw,
                final: finalTable
            };

            const response = await fetch(`${API_URL}/submit-draft`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(draftData)
            });

            if (!response.ok) {
                throw new Error('Failed to submit draft tracking');
            }

            return lesson_id; // Return lesson_id for future reference
        } catch (error) {
            console.error('Error tracking draft:', error);
            throw error;
        }
    }

    static async updateFinalTable(lesson_id, finalTableData) {
        try {
            const draftData = {
                lesson_id,
                input: '',  // Không cần cập nhật lại
                raw: '',    // Không cần cập nhật lại
                final: JSON.stringify(finalTableData)
            };

            const response = await fetch(`${API_URL}/submit-draft`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(draftData)
            });

            if (!response.ok) {
                throw new Error('Failed to update final table data');
            }
        } catch (error) {
            console.error('Error updating final table:', error);
            throw error;
        }
    }
}

export default TableDraftTracking;
```

Và cập nhật `generateQuestion.js` để sử dụng tracking:

```javascript:frontend/src/js/generateQuestion.js
import TableDraftTracking from './trackings/tableDraftTracking.js';

// ... existing code ...

async function handleGenerateClick() {
    const activeTab = document.querySelector('.tab-content.active');
    const isStandardForm = activeTab.id === 'standard-form';
    
    if (isStandardForm) {
        const inputData = {
            topic: document.getElementById('topic').value,
            level: document.getElementById('level').value,
            questionCount: document.getElementById('question-count').value,
            extraRequirements: document.getElementById('extra-requirements').value
        };
        
        const prompt = createGenerateQuestionPrompt();
        const response = await generateQuestions(prompt);
        
        // Track the generation
        try {
            const lesson_id = await TableDraftTracking.trackDraftGeneration(
                inputData,
                response
            );
            // Store lesson_id for later use when copying table
            window.currentLessonId = lesson_id;
        } catch (error) {
            console.error('Failed to track draft generation:', error);
        }
    } else {
        // Handle custom prompt case...
    }
}

// Modify copyTableToClipboard function
async function copyTableToClipboard(table) {
    // ... existing copy logic ...

    // After successful copy, update the final table data
    if (window.currentLessonId) {
        try {
            const tableData = Array.from(table.querySelectorAll('tbody tr')).map(row => {
                const cells = Array.from(row.cells);
                return {
                    question: cells[0].textContent,
                    structure: cells[1].textContent,
                    'main phrase': cells[2].textContent,
                    'optional phrase 1': cells[3].textContent,
                    'optional phrase 2': cells[4].textContent,
                    'question-vi': cells[5].textContent,
                    'structure-vi': cells[6].textContent,
                    'main phrase-vi': cells[7].textContent,
                    'optional phrase 1-vi': cells[8].textContent,
                    'optional phrase 2-vi': cells[9].textContent
                };
            });

            await TableDraftTracking.updateFinalTable(window.currentLessonId, tableData);
        } catch (error) {
            console.error('Failed to update final table data:', error);
        }
    }
    
    alert('Table copied to clipboard!');
}
```

Chức năng chính:
1. Tạo `lesson_id` theo format yêu cầu
2. Lưu trữ input data từ form
3. Lưu trữ raw response từ API
4. Cập nhật final table data khi người dùng copy table
5. Gửi tất cả dữ liệu tới backend thông qua `submit-draft` endpoint

Khi sử dụng:
1. Mỗi lần generate questions sẽ tạo một bản ghi mới với `input` và `raw`
2. Khi user copy table, sẽ cập nhật `final` cho bản ghi đó
3. Tất cả dữ liệu được lưu vào Larkbase thông qua `tableDraftController.js`

dữ liệu mẫu sẽ gửi đến tableDraftController.js là gì ?




Tôi sẽ cho bạn ví dụ cụ thể về dữ liệu được gửi từ `tableDraftTracking.js` đến `tableDraftController.js`:

```javascript
// Example data sent to /submit-draft endpoint

{
    "lesson_id": "business_meeting_2200_01112024",
    
    "input": "{
        \"topic\": \"Business Meeting\",
        \"level\": \"Intermediate\",
        \"questionCount\": \"5\",
        \"extraRequirements\": \"Focus on professional vocabulary\"
    }",
    
    "raw": "[
        {
            \"question\": \"Which company are you working for?\",
            \"structure\": \"I'm the ____ from ABC Company.\",
            \"main phrase\": \"Sales representative\",
            \"optional phrase 1\": \"Sales director\",
            \"optional phrase 2\": \"Sales associate\",
            \"question-vi\": \"Bạn đang làm việc cho công ty nào vậy?\",
            \"structure-vi\": \"Tôi là ____ từ công ty ABC.\",
            \"main phrase-vi\": \"Đại diện kinh doanh\",
            \"optional phrase 1-vi\": \"Giám đốc kinh doanh\",
            \"optional phrase 2-vi\": \"Nhân viên bán hàng\"
        },
        // ... more questions
    ]",
    
    "final": "[
        {
            \"question\": \"Which company are you working for?\",
            \"structure\": \"I'm the ____ from ABC Company.\",
            \"main phrase\": \"Sales representative\",
            \"optional phrase 1\": \"Sales director\",
            \"optional phrase 2\": \"Sales associate\",
            \"question-vi\": \"Bạn đang làm việc cho công ty nào vậy?\",
            \"structure-vi\": \"Tôi là ____ từ công ty ABC.\",
            \"main phrase-vi\": \"Đại diện kinh doanh\",
            \"optional phrase 1-vi\": \"Giám đốc kinh doanh\",
            \"optional phrase 2-vi\": \"Nhân viên bán hàng\"
        },
        // ... edited/final questions after user modifications
    ]"
}
```

Giải thích từng trường:

1. **lesson_id**: 
   - Format: `topic_time_date`
   - Example: `business_meeting_2200_01112024`

2. **input**: JSON string chứa thông tin form nhập
   ```json
   {
       "topic": "Business Meeting",
       "level": "Intermediate",
       "questionCount": "5",
       "extraRequirements": "Focus on professional vocabulary"
   }
   ```

3. **raw**: JSON string chứa response gốc từ API
   ```json
   [
       {
           "question": "Which company are you working for?",
           "structure": "I'm the ____ from ABC Company.",
           "main phrase": "Sales representative",
           // ... other fields
       },
       // ... more questions
   ]
   ```

4. **final**: JSON string chứa dữ liệu cuối cùng sau khi user edit và copy
   ```json
   [
       {
           "question": "Which company are you working for?",
           "structure": "I'm the ____ from ABC Company.",
           "main phrase": "Sales representative",
           // ... other fields
       },
       // ... final edited questions
   ]
   ```

Lưu ý:
- Tất cả các trường object/array đều được chuyển thành JSON string
- `final` có thể khác với `raw` nếu user đã chỉnh sửa câu hỏi
- Dữ liệu được gửi trong 2 thời điểm:
  1. Khi generate questions (`input` và `raw`)
  2. Khi copy table (cập nhật `final`)



Lỗi này xảy ra do vấn đề với JSON string. Hãy sửa lại cách xử lý dữ liệu trong `tableDraftTracking.js`:

```javascript:frontend/src/js/trackings/tableDraftTracking.js
import { config } from '../config.js';

const API_URL = config.production.apiUrl;

class TableDraftTracking {
    static generateLessonId(topic) {
        const now = new Date();
        const time = now.getHours().toString().padStart(2, '0') + 
                    now.getMinutes().toString().padStart(2, '0');
        const date = now.getDate().toString().padStart(2, '0') + 
                    (now.getMonth() + 1).toString().padStart(2, '0') + 
                    now.getFullYear();
        
        const normalizedTopic = topic.toLowerCase()
            .replace(/[^a-z0-9]/g, '_')
            .replace(/_+/g, '_');
            
        return `${normalizedTopic}_${time}_${date}`;
    }

    static async trackDraftGeneration(inputData, rawResponse, finalTable = '') {
        try {
            const lesson_id = this.generateLessonId(inputData.topic);
            
            // Clean and prepare data
            const cleanInput = {
                topic: inputData.topic || '',
                level: inputData.level || '',
                questionCount: inputData.questionCount || '',
                extraRequirements: inputData.extraRequirements || ''
            };

            // Ensure data is properly stringified
            const draftData = {
                lesson_id: lesson_id,
                input: JSON.stringify(cleanInput),
                raw: JSON.stringify(rawResponse),
                final: finalTable ? JSON.stringify(finalTable) : ''
            };

            const response = await fetch(`${API_URL}/submit-draft`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(draftData)
            });

            if (!response.ok) {
                throw new Error('Failed to submit draft tracking');
            }

            return lesson_id;
        } catch (error) {
            console.error('Error tracking draft:', error);
            throw error;
        }
    }

    static async updateFinalTable(lesson_id, finalTableData) {
        try {
            // Clean the table data before sending
            const cleanTableData = finalTableData.map(row => ({
                question: row.question || '',
                structure: row.structure || '',
                'main phrase': row['main phrase'] || '',
                'optional phrase 1': row['optional phrase 1'] || '',
                'optional phrase 2': row['optional phrase 2'] || '',
                'question-vi': row['question-vi'] || '',
                'structure-vi': row['structure-vi'] || '',
                'main phrase-vi': row['main phrase-vi'] || '',
                'optional phrase 1-vi': row['optional phrase 1-vi'] || '',
                'optional phrase 2-vi': row['optional phrase 2-vi'] || ''
            }));

            const draftData = {
                lesson_id,
                input: '',  // Không cần cập nhật
                raw: '',    // Không cần cập nhật
                final: JSON.stringify(cleanTableData)
            };

            const response = await fetch(`${API_URL}/submit-draft`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(draftData)
            });

            if (!response.ok) {
                throw new Error('Failed to update final table data');
            }
        } catch (error) {
            console.error('Error updating final table:', error);
            throw error;
        }
    }
}

export default TableDraftTracking;
```

Những thay đổi chính:
1. Thêm clean data trước khi stringify
2. Xử lý các trường hợp undefined/null
3. Đảm bảo tất cả dữ liệu được stringify đúng cách
4. Thêm kiểm tra và xử lý lỗi chi tiết hơn

Dữ liệu gửi đi sẽ có dạng:
```javascript
{
    "lesson_id": "business_meeting_2200_01112024",
    "input": "{\"topic\":\"Business Meeting\",\"level\":\"Intermediate\",\"questionCount\":\"5\",\"extraRequirements\":\"Focus on professional vocabulary\"}",
    "raw": "[{\"question\":\"Which company...\"}]",
    "final": "[{\"question\":\"Which company...\"}]"
}
```

Lưu ý:
- Tất cả các string JSON phải được escape đúng cách
- Không được có ký tự control trong string
- Đảm bảo dữ liệu được làm sạch trước khi stringify

Có vẻ như bạn đang lưu data vào larkbase ngay khi user generateQuestion hả. 
Tôi muốn khi user nhấn Copy mới lưu vào larkbase (TỨC LÀ CALL QUA: api/submit-draft)





Đúng rồi, tôi sẽ sửa lại để chỉ lưu data vào Larkbase khi user nhấn Copy:

```javascript:frontend/src/js/generateQuestion.js
// ... existing imports and code ...

let storagedLessons;
let inputDataTemp; // Thêm biến tạm để lưu inputData

async function handleGenerateClick() {
    const activeTab = document.querySelector('.tab-content.active');
    const isStandardForm = activeTab.id === 'standard-form';

    try {
        if (isStandardForm) {
            // Chỉ lưu tạm inputData, không gửi ngay
            inputDataTemp = {
                topic: document.getElementById('topic').value,
                level: document.getElementById('level').value,
                questionCount: document.getElementById('question-count').value,
                extraRequirements: document.getElementById('extra-requirements').value
            };

            const prompt = createGenerateQuestionPrompt();
            
            showLoadingDialog();
            const response = await fetch(`${API_URL}/generate-questions`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ prompt })
            });
            
            const data = await response.json();
            await processApiResponse(data);

        } else {
            const prompt = document.getElementById('custom-prompt-text').value;
            generateQuestions(prompt);
        }
    } catch (error) {
        console.error('Error in handleGenerateClick:', error);
        alert(error.message);
    } finally {
        hideLoadingDialog();
    }
}

async function copyTableToClipboard(table) {
    try {
        const tempTable = document.createElement('table');
        const tbody = document.createElement('tbody');
        const rows = table.querySelectorAll('tbody tr');
        
        // Copy table logic...
        rows.forEach(row => {
            const newRow = document.createElement('tr');
            for (let i = 0; i < row.cells.length - 2; i++) {
                const cell = row.cells[i].cloneNode(true);
                newRow.appendChild(cell);
            }
            tbody.appendChild(newRow);
        });
        
        tempTable.appendChild(tbody);
        
        // Copy to clipboard...
        tempTable.style.position = 'absolute';
        tempTable.style.left = '-9999px';
        document.body.appendChild(tempTable);
        
        const range = document.createRange();
        range.selectNode(tempTable);
        window.getSelection().removeAllRanges();
        window.getSelection().addRange(range);
        document.execCommand('copy');
        window.getSelection().removeAllRanges();
        
        document.body.removeChild(tempTable);

        // Extract table data
        const tableData = Array.from(rows).map(row => ({
            question: row.cells[0].textContent,
            structure: row.cells[1].textContent,
            'main phrase': row.cells[2].textContent,
            'optional phrase 1': row.cells[3].textContent,
            'optional phrase 2': row.cells[4].textContent,
            'question-vi': row.cells[5].textContent,
            'structure-vi': row.cells[6].textContent,
            'main phrase-vi': row.cells[7].textContent,
            'optional phrase 1-vi': row.cells[8].textContent,
            'optional phrase 2-vi': row.cells[9].textContent
        }));

        // Generate lesson_id and submit to Larkbase
        const lesson_id = TableDraftTracking.generateLessonId(inputDataTemp.topic);
        
        // Submit all data to Larkbase
        await TableDraftTracking.trackDraftGeneration(
            inputDataTemp,           // Input data
            storagedLessons,        // Raw API response
            tableData               // Final edited table
        );

        console.log('Data submitted to Larkbase with lesson_id:', lesson_id);
        alert('Table copied to clipboard!');

    } catch (error) {
        console.error('Error in copyTableToClipboard:', error);
        alert('Error copying table: ' + error.message);
    }
}

// ... rest of your code ...

export { 
    handleGenerateClick,
    storagedLessons,
    generateQuestions,
    processApiResponse
};
```


Và cập nhật `tableDraftTracking.js`:

```javascript:frontend/src/js/trackings/tableDraftTracking.js
class TableDraftTracking {
    static generateLessonId(topic) {
        const now = new Date();
        const time = now.getHours().toString().padStart(2, '0') + 
                    now.getMinutes().toString().padStart(2, '0');
        const date = now.getDate().toString().padStart(2, '0') + 
                    (now.getMonth() + 1).toString().padStart(2, '0') + 
                    now.getFullYear();
        
        const normalizedTopic = topic.toLowerCase()
            .replace(/[^a-z0-9]/g, '_')
            .replace(/_+/g, '_');
            
        return `${normalizedTopic}_${time}_${date}`;
    }

    static async trackDraftGeneration(inputData, rawResponse, finalTable) {
        try {
            const lesson_id = this.generateLessonId(inputData.topic);
            
            // Prepare data for Larkbase
            const draftData = {
                lesson_id: lesson_id,
                input: JSON.stringify(inputData),
                raw: JSON.stringify(rawResponse),
                final: JSON.stringify(finalTable)
            };

            const response = await fetch(`${API_URL}/submit-draft`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(draftData)
            });

            if (!response.ok) {
                throw new Error('Failed to submit draft tracking');
            }

            return lesson_id;
        } catch (error) {
            console.error('Error tracking draft:', error);
            throw error;
        }
    }
}

export default TableDraftTracking;
```


Những thay đổi chính:
1. Loại bỏ việc gửi data khi generate
2. Thêm biến `inputDataTemp` để lưu tạm thông tin input
3. Gửi tất cả data (input, raw, final) khi user nhấn Copy
4. Đơn giản hóa `TableDraftTracking` class
5. Chỉ còn một lần call API khi copy

Flow mới:
1. User nhập form và generate -> Lưu tạm data
2. User chỉnh sửa nếu cần
3. User nhấn Copy -> Gửi tất cả data vào Larkbase

- raw sẽ là cái mà version đầu tiên khi user nhấn generateQuestion (tức là chưa bị user sửa, xóa, - 2 action del, edit) 
- final là cái version cuối khi user nhấn Copy
------------
Có vẻ bạn đang hơi nhầm gì đó với raw? 
Hông sao, check lại giùm tui nha


Ví dụ 

_i_ch_i_m_0158_01112024	
input

{"topic":"Đi chơi đêm","level":"Below A1","questionCount":"3","extraRequirements":""}	

raw
[{"question":"What do you like to do at night?","structure":"I like to go ____ at night.","main phrase":"for a walk","optional phrase 1":"to the park","optional phrase 2":"to the movies","question-vi":"Bạn thích làm gì vào ban đêm?","structure-vi":"Tôi thích đi ____ vào ban đêm.","main phrase-vi":"đi dạo","optional phrase 1-vi":"đến công viên","optional phrase 2-vi":"đến rạp chiếu phim"},{"question":"Where do you want to go tonight?","structure":"I want to go ____ tonight.","main phrase":"to the beach","optional phrase 1":"to a restaurant","optional phrase 2":"to a party","question-vi":"Bạn muốn đi đâu tối nay?","structure-vi":"Tôi muốn đi ____ tối nay.","main phrase-vi":"đến bãi biển","optional phrase 1-vi":"đến nhà hàng","optional phrase 2-vi":"đến bữa tiệc"},{"question":"Who do you go out with at night?","structure":"I go out with my ____ at night.","main phrase":"friends","optional phrase 1":"family","optional phrase 2":"classmates","question-vi":"Bạn đi chơi với ai vào ban đêm?","structure-vi":"Tôi đi chơi với ____ vào ban đêm.","main phrase-vi":"bạn bè","optional phrase 1-vi":"gia đình","optional phrase 2-vi":"bạn học"}]

final
[{"question":"Who do you go out with at night?","structure":"I go out with my ____ at night.","main phrase":"friends","optional phrase 1":"family","optional phrase 2":"classmates","question-vi":"Bạn đi chơi với ai vào ban đêm?","structure-vi":"Tôi đi chơi với ____ vào ban đêm.","main phrase-vi":"bạn bè","optional phrase 1-vi":"gia đình","optional phrase 2-vi":"bạn học"}]








Ah, tôi hiểu rồi! Trong ví dụ của bạn:

1. `raw` có 3 câu hỏi (version đầu tiên sau khi generate)
2. `final` chỉ còn 1 câu (sau khi user đã xóa 2 câu và copy)

Hãy sửa lại code để đảm bảo điều này:

```javascript:frontend/src/js/generateQuestion.js
let storagedLessons;
let inputDataTemp;
let rawResponseTemp; // Sẽ luôn giữ version đầu tiên với đầy đủ câu hỏi

async function handleGenerateClick() {
    const activeTab = document.querySelector('.tab-content.active');
    const isStandardForm = activeTab.id === 'standard-form';

    try {
        if (isStandardForm) {
            inputDataTemp = {
                topic: document.getElementById('topic').value,
                level: document.getElementById('level').value,
                questionCount: document.getElementById('question-count').value,
                extraRequirements: document.getElementById('extra-requirements').value
            };

            const prompt = createGenerateQuestionPrompt();
            
            showLoadingDialog();
            const response = await fetch(`${API_URL}/generate-questions`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ prompt })
            });
            
            const data = await response.json();
            
            const processedData = await processApiResponse(data);
            rawResponseTemp = [...processedData]; // Lưu version đầu tiên với đầy đủ câu hỏi
            
            storagedLessons = processedData; // Version này sẽ thay đổi khi user edit/delete

        } else {
            const prompt = document.getElementById('custom-prompt-text').value;
            generateQuestions(prompt);
        }
    } catch (error) {
        console.error('Error in handleGenerateClick:', error);
        alert(error.message);
    } finally {
        hideLoadingDialog();
    }
}

function deleteLesson(index, lessons) {
    // Chỉ xóa từ storagedLessons, không động đến rawResponseTemp
    lessons.splice(index, 1);
    displayGeneratedQuestions(lessons);
}

async function copyTableToClipboard(table) {
    try {
        // ... copy table logic ...

        // Lấy data hiện tại của table (sau khi đã edit/delete)
        const finalTableData = Array.from(table.querySelectorAll('tbody tr')).map(row => ({
            question: row.cells[0].textContent,
            structure: row.cells[1].textContent,
            'main phrase': row.cells[2].textContent,
            'optional phrase 1': row.cells[3].textContent,
            'optional phrase 2': row.cells[4].textContent,
            'question-vi': row.cells[5].textContent,
            'structure-vi': row.cells[6].textContent,
            'main phrase-vi': row.cells[7].textContent,
            'optional phrase 1-vi': row.cells[8].textContent,
            'optional phrase 2-vi': row.cells[9].textContent
        }));

        // Submit to Larkbase
        await TableDraftTracking.trackDraftGeneration(
            inputDataTemp,      // Form data
            rawResponseTemp,    // Version đầu tiên với đầy đủ câu hỏi
            finalTableData      // Version cuối (có thể ít câu hỏi hơn do đã xóa)
        );

        console.log('Data submitted to Larkbase:', {
            input: inputDataTemp,
            raw: rawResponseTemp,    // Full version
            final: finalTableData    // Edited/deleted version
        });
        
        alert('Table copied to clipboard!');

    } catch (error) {
        console.error('Error in copyTableToClipboard:', error);
        alert('Error copying table: ' + error.message);
    }
}
```



Điểm quan trọng:
1. `rawResponseTemp` sẽ luôn giữ version đầu tiên với đầy đủ câu hỏi
2. `storagedLessons` sẽ thay đổi khi user edit/delete
3. Khi delete, chỉ xóa từ `storagedLessons`
4. Khi copy:
   - `raw`: lấy từ `rawResponseTemp` (đầy đủ câu hỏi)
   - `final`: lấy từ table hiện tại (có thể ít câu hơn)

Ví dụ flow:
1. Generate 3 câu → `rawResponseTemp` = 3 câu
2. User xóa 2 câu → `storagedLessons` = 1 câu
3. User copy → Submit:
   - `raw`: 3 câu (từ `rawResponseTemp`)
   - `final`: 1 câu (từ table hiện tại)